In [11]:
!pip3 install --ugprade keras-preprocessing


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --ugprade


In [ ]:
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def process(X,file):
    img=cv2.imread(file)
    X.append(img)

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim2'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        TVT[i]+=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        Y_TVT[i]+=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

train_datagen =tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rescale=1./255,
        color_mode="grayscale",
        zoom_range=[-0.2,0.2],
        rotation_range=[-20,20],
        horizontal_flip=True)
validation_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
        color_mode="grayscale")
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
        color_mode="grayscale")

train_datagen.fit(Train)
validation_datagen.fit(Valid)
test_datagen.fit(Test)

def train(epochs_num,batch_len,conv_neurons,art_neurons):
    
    def mlp_model():
        Inp=layers.Input(shape=(400,400,1))
        hidden=Inp
        for conv_neuron in conv_neurons:
            hidden=layers.Conv2D(conv_neurons,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            i2=i+1
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(0.0001))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,150,[32,64,128],[50,50])



   
        
